https://huggingface.co/docs/peft/quicktour

关闭多线程分词，避免多进程冲突警告。

Hugging Face 的 tokenizers 库在后台使用 Rust 多线程加速（并行分词）。
但是，当 Python 的 multiprocessing 在已经启用了并行的情况下调用 fork() 时，会触发这个警告，以避免潜在的死锁。

In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"  # 禁用分词器并行警告
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"  # 允许 MPS 不支持的算子自动 fallback 到 CPU

In [2]:
import torch
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [3]:
import numpy as np

## 一、使用 bigscience/mt0-large

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "bigscience/mt0-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model = model.to(device)

inputs = tokenizer("Translate English to Chinese: Hello, how are you?", return_tensors="pt").to(device)
outputs = model.generate(**inputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

嗨,你怎麼樣?


## 二、微调训练

每个 PEFT 方法都由一个 PeftConfig 类定义，该类存储构建 PeftModel 所需的所有重要参数。

例如，要使用 LoRa 进行训练，请加载并创建一个 LoraConfig 类。

- task_type ：要训练的任务（本例中为序列到序列语言建模）
- inference_mode ：是否使用该模型进行推理
- r ：低秩矩阵的维数
- lora_alpha ：低秩矩阵的缩放因子
- lora_dropout ：LoRa 层的 dropout 概率

In [6]:
from peft import LoraConfig, TaskType
peft_config = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM,  inference_mode=False, r=4, lora_alpha=8, lora_dropout=0.05)

LoraConfig 设置完成后，使用 get_peft_model() 函数创建一个 PeftModel 。

该函数接受一个基础模型（可以从 Transformers 库加载）和一个 LoraConfig， 其中包含用于配置模型以使用 LoRA 进行训练的参数。

In [7]:
from peft import get_peft_model

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 442,368 || all params: 582,843,648 || trainable%: 0.0759


现在你可以使用 Transformers Trainer 、Accelerate 或任何自定义的 PyTorch 训练循环来训练模型了。

例如，要使用 Trainer 类进行训练，请设置一个 TrainingArguments 类，并设置一些训练超参数。

加载一个翻译数据集，英文到中文

In [8]:
from datasets import load_dataset
dataset = load_dataset("opus100", "en-zh", split="train[:1%]").train_test_split(test_size=0.1)

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 9000
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 1000
    })
})

In [10]:
dataset["train"][:3]

{'translation': [{'en': 'Main professional activities', 'zh': '主要专业活动'},
  {'en': 'At the same meeting, the Commission adopted the draft report on its eighth session and entrusted the Rapporteur with its completion.',
   'zh': '2. 在同次会议上，委员会通过了其第八届会议报告草稿，并委托报告员完成此项工作。'},
  {'en': 'K0581557 xx0505', 'zh': '为节省开支，本文件仅作少量印发。'}]}

对数据进行预处理

In [11]:
# 定义预处理函数：把原始文本构造成给模型的 prompt，并进行分词
def preprocess(examples):
    # 构造输入 prompt（mt0 是 instruction-style，因此加上任务说明）   
    inputs = [f"translate English to Chinese: {ex['en']}" for ex in examples["translation"]] # 把每个英文句子包装成翻译指令
    targets = [ex["zh"] for ex in examples["translation"]]

    # 使用 tokenizer 对 inputs 做编码（截断 + padding 到 max_length）
    model_inputs = tokenizer(inputs, truncation=True, padding="max_length", max_length=128)  # 编码输入，统一长度

    # 对目标文本进行编码，得到 labels
    labels = tokenizer(targets, truncation=True, padding="max_length", max_length=128)  # 编码目标文本

    # 将 labels 的 pad token 替换为 -100，这是 transformers 损失函数忽略的标记
    # 这样在计算交叉熵时，对 pad 部分不会贡献损失
    labels_input_ids = [
        [(token if token != tokenizer.pad_token_id else -100) for token in label] for label in labels["input_ids"]
    ]  # 将 labels 中的 pad id 转为 -100

    # 把处理好的 labels 放回 model_inputs 字典，Trainer 会读取 "labels" 字段用于计算 loss
    model_inputs["labels"] = labels_input_ids  # 为返回的数据添加 labels

    # 返回最终的编码后的输入字典（包含 input_ids, attention_mask, labels）
    return model_inputs  # 返回每个样本的字典

# 把预处理函数映射到整个数据集（batched=True 表示按批次处理以加速）
tokenized_datasets = dataset.map(preprocess, batched=True)  # 对 dataset 的每个 split 应用 preprocess 函数

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [12]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 9000
    })
    test: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
})

In [13]:
example = tokenized_datasets["train"][0]
print("Input:", tokenizer.decode(example["input_ids"], skip_special_tokens=True))
print("Label:", tokenizer.decode([id for id in example["labels"] if id != -100], skip_special_tokens=True))

Input: translate English to Chinese: Main professional activities
Label: 主要专业活动


加载评测指标

In [14]:
import evaluate  # Hugging Face evaluate 库，用于加载评测指标
# 加载评测指标（示例使用 sacrebleu 用于机器翻译）
metric = evaluate.load("sacrebleu")  # 加载 sacrebleu 指标用于 BLEU 风格评测

In [15]:
# 定义 compute_metrics，用于 Trainer 在 eval 时计算指标
def compute_metrics(eval_pred):
    # eval_pred 的典型形式是 (predictions, labels)；这里 predictions 是生成器输出的 token id 矩阵
    preds, labels = eval_pred  # 解包预测结果和标签

    # 如果 predictions 是 logits（非生成模式），则需要取 argmax；但 predict_with_generate=True 时 preds 已是 token id
    if isinstance(preds, tuple):  # 万一 preds 是 tuple（某些版本返回 logits 等），我们取第一个
        preds = preds[0]

    # 将预测中的 -100（如果有的话）替换为 tokenizer.pad_token_id，便于 decode
    # 但正常生成结果里不应该有 -100，这里是稳健处理
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)  # 将 -100 换为 pad id（以便 decode）

    # 把预测和真实标签解码成字符串（跳过 special tokens）
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)  # 解码预测文本
    # labels 里仍可能有 -100，需要把它们换成 pad_token_id 再解码
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)  # 将 labels 中的 -100 替回 pad id
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)  # 解码真实文本

    # sacrebleu 的 compute 需要 references 是 list of list 的形式：[[ref1], [ref2], ...]
    references = [[l] for l in decoded_labels]  # 为每个样本创建单一引用列表

    # 调用 sacrebleu 指标进行计算并返回结果字典（sacrebleu 返回 'score' 键表示 BLEU）
    result = metric.compute(predictions=decoded_preds, references=references)  # 计算 sacrebleu 分数
    # 将结果包装成 dict 返回，Trainer 会把它显示在 eval 输出里
    return {"bleu": result["score"]}  # 返回一个包含 BLEU 分数的字典

训练参数

In [37]:
from transformers import Seq2SeqTrainingArguments
# ---- 训练参数 ----
training_args = Seq2SeqTrainingArguments(
    output_dir="./mt0-lora",                  # 模型输出和检查点保存目录
    per_device_train_batch_size=1,            # 每个 GPU/设备上的训练 batch size（根据显存调整）, # Mac上必须非常小，否则内存爆
    per_device_eval_batch_size=1,             # 验证时的 batch size
    gradient_accumulation_steps=8,            # # 模拟 batch size=8
    learning_rate=5e-5,                       # 学习率，默认5e-5，内存小调低学习率
    num_train_epochs=1,                       # 训练轮次，#先跑1个epoch测试稳定性
    eval_strategy="epoch",                    # 评估频率（这里设置为每个 epoch 评估一次）
    predict_with_generate=True,               # 在评估时使用 generate() 来产生文本（必需用于语言生成任务）
    save_strategy="epoch",                    # epoch 表示每个 epoch 保存一次checkpoint），# no 不保存中间检查点，节省内存
    logging_strategy="steps",                 # 日志策略（按 steps）
    logging_steps=10,                         # 每多少 step 打印一次日志
    fp16=False,                               # 如果支持则开启半精度训练以节省显存（需要 CUDA + 支持）#  MPS 不支持 fp16
    load_best_model_at_end=True,              # 在训练结束加载最佳验证结果对应的模型
    dataloader_pin_memory=False,              # 在 CUDA 上能让 DataLoader 更快地把数据拷贝到 GPU。MPS 不支持
    metric_for_best_model="bleu",             # 根据哪个指标判断最佳模型
    report_to="none",                     # ✅ 不向wandb等发送日志
    log_level="info",                     # ✅ 输出日志到控制台
    log_level_replica="info",             # ✅ 对多进程安全
    disable_tqdm=False,
)

PyTorch: setting up devices


In [38]:
import logging
logging.getLogger("transformers.trainer").setLevel(logging.INFO)

创建训练器

In [39]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,                              # 注入（已包装 LoRA 的）模型
    args=training_args,                       # 训练参数
    train_dataset=tokenized_datasets["train"],# 训练集
    eval_dataset=tokenized_datasets["validation"] if "validation" in tokenized_datasets else tokenized_datasets["test"],  # 优先使用 validation split，否则用 test
    processing_class=tokenizer,     # tokenizer（用于 decode/generation 时）
    compute_metrics=compute_metrics,          # 评估时要计算的指标函数
)


In [40]:
import transformers
transformers.__version__

'4.57.1'

In [42]:
from transformers.trainer_callback import PrinterCallback
trainer.add_callback(PrinterCallback)

In [43]:
# ---- 启动训练 ----
trainer.train()  # 运行训练过程；训练期间会按 training_args 指定的策略保存 checkpoint 并在 eval 时计算指标

The following columns in the Training set don't have a corresponding argument in `PeftModelForSeq2SeqLM.forward` and have been ignored: translation. If translation are not expected by `PeftModelForSeq2SeqLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 9,000
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 8
  Total optimization steps = 1,125
  Number of trainable parameters = 442,368


Epoch,Training Loss,Validation Loss,Bleu
1,3.815500,3.143986,9.219724


{'loss': 3.744, 'grad_norm': 0.9939719438552856, 'learning_rate': 4.96e-05, 'epoch': 0.008888888888888889}
{'loss': 3.6232, 'grad_norm': 1.101561188697815, 'learning_rate': 4.915555555555556e-05, 'epoch': 0.017777777777777778}
{'loss': 3.7036, 'grad_norm': 0.9699844121932983, 'learning_rate': 4.871111111111111e-05, 'epoch': 0.02666666666666667}
{'loss': 3.6186, 'grad_norm': 1.1010091304779053, 'learning_rate': 4.826666666666667e-05, 'epoch': 0.035555555555555556}
{'loss': 3.6292, 'grad_norm': 1.0778459310531616, 'learning_rate': 4.782222222222222e-05, 'epoch': 0.044444444444444446}
{'loss': 3.4263, 'grad_norm': 1.2162610292434692, 'learning_rate': 4.737777777777778e-05, 'epoch': 0.05333333333333334}
{'loss': 3.5196, 'grad_norm': 0.8735532164573669, 'learning_rate': 4.6933333333333333e-05, 'epoch': 0.06222222222222222}
{'loss': 3.6469, 'grad_norm': 0.8126728534698486, 'learning_rate': 4.648888888888889e-05, 'epoch': 0.07111111111111111}
{'loss': 3.8267, 'grad_norm': 1.1234153509140015, 

The following columns in the Evaluation set don't have a corresponding argument in `PeftModelForSeq2SeqLM.forward` and have been ignored: translation. If translation are not expected by `PeftModelForSeq2SeqLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 1000
  Batch size = 1


{'eval_loss': 3.143986225128174, 'eval_bleu': 9.219723728891687, 'eval_runtime': 361.7814, 'eval_samples_per_second': 2.764, 'eval_steps_per_second': 2.764, 'epoch': 1.0}


Saving model checkpoint to ./mt0-lora/checkpoint-1125
loading configuration file config.json from cache at /Users/huhao/.cache/huggingface/hub/models--bigscience--mt0-base/snapshots/0c500042c1f80b6b6ec208cf73d9ee43f1dac245/config.json
Model config MT5Config {
  "architectures": [
    "MT5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "dtype": "float32",
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "mt5",
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Tokenizer",
  "transformers_version": "4.57.1",
  "us

{'train_runtime': 1833.2842, 'train_samples_per_second': 4.909, 'train_steps_per_second': 0.614, 'train_loss': 3.6257413109673395, 'epoch': 1.0}


TrainOutput(global_step=1125, training_loss=3.6257413109673395, metrics={'train_runtime': 1833.2842, 'train_samples_per_second': 4.909, 'train_steps_per_second': 0.614, 'train_loss': 3.6257413109673395, 'epoch': 1.0})

In [44]:
# 9. 保存 LoRA 权重
trainer.save_model("./mt0-lora-mac-final")

Saving model checkpoint to ./mt0-lora-mac-final
loading configuration file config.json from cache at /Users/huhao/.cache/huggingface/hub/models--bigscience--mt0-base/snapshots/0c500042c1f80b6b6ec208cf73d9ee43f1dac245/config.json
Model config MT5Config {
  "architectures": [
    "MT5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "dtype": "float32",
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "mt5",
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Tokenizer",
  "transformers_version": "4.57.1",
  "use_cach

# 测试生成的模型

In [45]:
# 加载微调后模型
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import PeftModel

model_name = "bigscience/mt0-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model = PeftModel.from_pretrained(base_model, "./mt0-lora-mac-final")
model = model.to(device)

loading file spiece.model from cache at /Users/huhao/.cache/huggingface/hub/models--bigscience--mt0-base/snapshots/0c500042c1f80b6b6ec208cf73d9ee43f1dac245/spiece.model
loading file tokenizer.json from cache at /Users/huhao/.cache/huggingface/hub/models--bigscience--mt0-base/snapshots/0c500042c1f80b6b6ec208cf73d9ee43f1dac245/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /Users/huhao/.cache/huggingface/hub/models--bigscience--mt0-base/snapshots/0c500042c1f80b6b6ec208cf73d9ee43f1dac245/special_tokens_map.json
loading file tokenizer_config.json from cache at /Users/huhao/.cache/huggingface/hub/models--bigscience--mt0-base/snapshots/0c500042c1f80b6b6ec208cf73d9ee43f1dac245/tokenizer_config.json
loading file chat_template.jinja from cache at None
loading configuration file config.json from cache at /Users/huhao/.cache/huggingface/hub/models--bigscience--mt0-base/snapshots/0c500042c1f80b6b6ec208cf73d9ee43f1dac245/config.js

In [47]:
# 测试生成
text = '''Translate English to Chinese: 
The difficult access to these older sites for displaced persons gives rise to 
concerns about the living conditions of the people who are residents there, 
in particular widows, the elderly and children, 
who often live in intolerable hardship.'''
inputs = tokenizer(, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=1024)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


对于那些被转移的人来说,这些老城的人口对他们居住的状况有很大影响,尤其是老人、老人和孩子,他们经常生活在极其困难的困难之中。
